#### Generate references with GPT4o and Llama3-70b

#### GPT-4o

In [ ]:
# CONFIGURATION
import pandas as pd
import time
import os
from openai import AzureOpenAI

# Get the base directory dynamically 
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__ if '__file__' in globals() else '.')))
EVAL_DIR = os.path.join(BASE_DIR, "eval")

print(f"Base directory: {BASE_DIR}")
print(f"Evaluation directory: {EVAL_DIR}")

# Initialize the OpenAI client with environment variables for security
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

if not azure_api_key:
    raise ValueError("AZURE_OPENAI_API_KEY environment variable is required")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version
)

print("Azure OpenAI client initialized successfully")
print("\nEnvironment Variables Needed:")
print("- AZURE_OPENAI_API_KEY: Your Azure OpenAI API key (required)")
print("- AZURE_OPENAI_ENDPOINT: Your Azure OpenAI endpoint (optional, defaults to cmac-openai-default)")
print("- AZURE_OPENAI_API_VERSION: Your Azure OpenAI API version (optional, defaults to 2024-02-15-preview)")

# Read the random_prompts.csv file with dynamic path
csv_file_path = os.path.join(EVAL_DIR, 'updated_random_prompts.csv')
random_prompts = pd.read_csv(csv_file_path)

# Add new columns for the generated text and processing time
random_prompts['GPT-4o'] = None
random_prompts['Processing Time (GPT-4o)'] = None

try:
    # Iterate through each prompt
    for i, row in random_prompts.iterrows():      
        prompt = row['Combined Text']
        print(f"Processing prompt {i + 1} out of {len(random_prompts)}")

        start_time = time.time()
        response = client.chat.completions.create(
            model="gpt-4o-default",  # model = "deployment_name".
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        end_time = time.time()
        processing_time = end_time - start_time

        # Update the DataFrame with the response and processing time
        random_prompts.at[i, 'GPT-4o'] = response.choices[0].message.content
        random_prompts.at[i, 'Processing Time (GPT-4o)'] = processing_time

        print(f"Query LLM processing time: {processing_time}")
        print(f"Model's Answer for prompt {i + 1}:\n{response.choices[0].message.content}\n")

        # Save the updated DataFrame back to a CSV file after each prompt
        random_prompts.to_csv('updated_random_prompts.csv', index=False)

except Exception as e:
    print(f"An error occurred: {e}")

####  Llama3-70b

In [ ]:
import pandas as pd
import time
import os
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key="key"
)

# Add new columns for the generated text and processing time 
random_prompts['llama3-70b'] = None
random_prompts['Processing Time (llama3-70b)'] = None

try:
    # Iterate through each prompt
    for i, row in random_prompts.iterrows():
        prompt = row['Combined Text']
        print(f"Processing prompt {i + 1} out of {len(random_prompts)}")

        start_time = time.time()
        llm = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            model="llama3-70b-8192",
        )
        end_time = time.time()
        processing_time = end_time - start_time

        # Update the DataFrame with the response and processing time
        random_prompts.at[i, 'llama3-70b'] = llm.choices[0].message.content
        random_prompts.at[i, 'Processing Time (llama3-70b)'] = processing_time

        print(f"Query LLM processing time: {processing_time}")
        print(f"Model's Answer for prompt {i + 1}:\n{llm.choices[0].message.content}\n")

        # Save the updated DataFrame back to a CSV file after each prompt
        random_prompts.to_csv('updated_random_prompts.csv', index=False)

except Exception as e:
    print(f"An error occurred: {e}")

#### 100(150) prompts triples (vicuna, llama3,mistral)

In [ ]:
# filtering prompts during annotation, throw bad ones, add new ones
# random_prompts.csv
import pandas as pd
# Read the random_prompts.csv file
csv_file_path = 'updated_random_prompts.csv'
random_prompts = pd.read_csv(csv_file_path)

In [ ]:
# Initialize the LLM model
llm = Ollama(model="mistral:instruct")

try:
    for i, row in random_prompts.iloc[128:].iterrows():
        prompt = row['Combined Text'] 
        print(f"Processing prompt {i + 1} out of {len(random_prompts)}")
        
        start_time = time.time()
        response = llm.complete(prompt)
        end_time = time.time()
        processing_time = end_time - start_time
        
        # Update the DataFrame with the response and processing time
        random_prompts.at[i, 'mistral:instruct'] = response
        random_prompts.at[i, 'Processing Time (mistral)'] = processing_time
        
        print(f"Query LLM processing time: {processing_time}")
        print(f"Model's Answer for prompt {i + 1}:\n{response}\n")
        
        # Save the updated DataFrame back to a CSV file after each prompt
        random_prompts.to_csv('updated_random_prompts.csv', index=False)


except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Initialize the LLM model
llm = Ollama(model="vicuna")

# Add new columns for the generated text and processing time
random_prompts['vicuna'] = None
random_prompts['Processing Time (vicuna) '] = None

try:
    for i, row in random_prompts.iterrows():
        prompt = row['Combined Text'] 
        print(f"Processing prompt {i + 1} out of {len(random_prompts)}")
        
        start_time = time.time()
        response = llm.complete(prompt)
        end_time = time.time()
        processing_time = end_time - start_time
        
        # Update the DataFrame with the response and processing time
        random_prompts.at[i, 'vicuna'] = response
        random_prompts.at[i, 'Processing Time (vicuna)'] = processing_time
        
        print(f"Query LLM processing time: {processing_time}")
        print(f"Model's Answer for prompt {i + 1}:\n{response}\n")
        
        # Save the updated DataFrame back to a CSV file after each prompt
        random_prompts.to_csv('updated_random_prompts.csv', index=False)

except Exception as e:
    print(f"An error occurred: {e}")
    

In [ ]:
# Initialize the LLM model
llm = Ollama(model="llama3")

# Add new columns for the generated text and processing time
random_prompts['llama3'] = None
random_prompts['Processing Time (llama3) '] = None

try:
    for i, row in random_prompts.iterrows():
        prompt = row['Combined Text'] 
        print(f"Processing prompt {i + 1} out of {len(random_prompts)}")
        
        start_time = time.time()
        response = llm.complete(prompt)
        end_time = time.time()
        processing_time = end_time - start_time
        
        # Update the DataFrame with the response and processing time
        random_prompts.at[i, 'llama3'] = response
        random_prompts.at[i, 'Processing Time (llama3)'] = processing_time
        
        print(f"Query LLM processing time: {processing_time}")
        print(f"Model's Answer for prompt {i + 1}:\n{response}\n")
        
        # Save the updated DataFrame back to a CSV file after each prompt
        random_prompts.to_csv('updated_random_prompts.csv', index=False)

except Exception as e:
    print(f"An error occurred: {e}")
    